In [3]:
import requests
import json
import numpy as np

In [22]:
url = "http://localhost:11434/api/embeddings"


def get_embed(prompt, model="mistral"):
    # Define the payload (data to be sent with the request)
    payload = {
        "model": model,
        "prompt": prompt
        }

    data = None
    # Send the POST request with streaming
    with requests.post(url, json=payload, stream=True) as response:
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Iterate through the streaming responses
            for line in response.iter_lines():
                if line:
                    # Parse the JSON response
                    data = json.loads(line.decode("utf-8"))
                    data = np.array(data["embedding"])

        else:
            print(f"Error: {response.status_code} - {response.text}")
    
    return data

In [31]:
SERVER_URL = "http://localhost:11434"

def generate_response(prompt, model='tinyllama', context=[]):
    API_END = "/api/generate"

    payload = {
        "model": model,
        "prompt": prompt,
        "context": context
    }

    url = SERVER_URL + API_END
    next_context = []

    # Send the POST request with streaming
    with requests.post(url, json=payload, stream=True) as response:
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Iterate through the streaming responses
            for line in response.iter_lines():
                if line:
                    # Parse the JSON response
                    data = json.loads(line.decode("utf-8"))
                    
                    # Print without buffering
                    print(data["response"], end="", flush=True)
                    if data["done"]: 
                        next_context = data["context"]

        else:
            print(f"Error: {response.status_code} - {response.text}")

    return next_context